<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/score_survey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NPS

## Importing libraries

In [ ]:
# Upgrade Matplotlib
!pip install matplotlib --upgrade
#!pip install plotly --upgrade

In [ ]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [ ]:
# Storing loaded data from excel to a pandas dataframe
import io
df1 = pd.read_excel(io.BytesIO(uploaded1['b2c_rnps_original.xlsx']))

In [ ]:
# Storing loaded data from csv to a pandas dataframe
#import io
#df1 = pd.read_csv(io.BytesIO(uploaded1['Script_202209261147.csv']), sep='|', engine='python')

In [ ]:
# Chainging columns to lower case
df1.rename(columns={i:i.lower() for i in df1.columns}, inplace=True)

In [ ]:
# Selecting categorical and numeric columns
custom_list_1 = df1.select_dtypes(include=['object']).copy().columns.tolist()
custom_list_2 = df1.select_dtypes(include=['float64']).copy().columns.tolist()

In [ ]:
# Checking the dataframe info
df1.info()

## Exploring Data

In [ ]:
# Exploring list of categorical columns with value counts
@interact(Column_name=custom_list_1, Percentage=[True, False])
def explore_value_counts(Column_name, Percentage):
  df = df1.value_counts(subset=[Column_name], normalize=Percentage, dropna=False)
  return df

## Melting dataframe

In [ ]:
# Replacing null values with 'No answer' text
df1 = df1.fillna('No answer')

In [ ]:
# Replacing string answers with int values
ans_1 = {'6 ? Helt enig':6,'6 - Helt enig':6,'6 - Svært fornøyd':6,'6 ? Svært fornøyd':6,'6 - Veldig sannsynlig':6,
         '0 - Svært lite sannsynlig':0, '10 - Svært sannsynlig':10,
         '1 - Helt uenig':1,'1 ? Helt uenig':1,'1 ? Svært lite fornøyd':1,'1 - Svært lite fornøyd':1,
         '1 - Misfornøyd':1,'1 - Veldig lite sannsynlig':1,
         'Vet ikke / Ikke relevant': 'Vet ikke'}
df1 = df1.replace(to_replace=ans_1)

In [ ]:
# Replacing string answers with short text
ans_2 = {'Ja, det går fint':'Ja','Nei, jeg ønsker ikke å bli oppringt':'Nei','Ikke noe jeg bruker aktivt':'No active use'}
df1 = df1.replace(to_replace=ans_2)

In [ ]:
# Binning scale 0-10
bin_1 = {0:'Detractors',1:'Detractors',2:'Detractors',3:'Detractors',4:'Detractors',5:'Detractors',
         6:'Detractors',7:'Passive',8:'Passive',9:'Promoters',10:'Promoters'}
df1.iloc[:,6].replace(to_replace=bin_1, inplace=True)

In [ ]:
# Binning scale 1-6
bin_2 = {6:'Positive',5:'Positive',4:'Neutral',3:'Neutral',2:'Negative',1:'Negative'}
for i in range(7,19): #Range of columns, could it be dynamic?
  df1.iloc[:,i].replace(to_replace=bin_2, inplace=True)

In [ ]:
# Checking replace operations
df1.iloc[:,6].value_counts()

In [ ]:
# Melting dataframe to long format
df1_melted = pd.melt(df1, id_vars=['period','questionnaireid','answereddatetime','answerformid','domainkey'],
                     value_vars=[i for i in df1.columns[6:19]], #Range of columns, could it be dynamic?
                     var_name='question', value_name='answer', ignore_index=True)

In [ ]:
# Checking melted df
df1_melted.tail()

In [ ]:
# Grouping by answer to aggregate by period
df1_g = df1_melted.groupby(by=['period','question','answer']).agg(num_count = ('answerformid','count')).reset_index()

In [ ]:
# Checking grouped dataframe
df1_g.tail(10)

## Exporting results to local drive

In [ ]:
# Exporting main excel file
with pd.ExcelWriter('2b_rnps.xlsx', engine='openpyxl') as writer:
  df1_g.to_excel(writer, sheet_name='Sheet1', index=False)
  #df1_melted.to_excel(writer, sheet_name='Sheet2', index=False)
files.download('2b_rnps.xlsx')

In [ ]:
# Exporting list of excel sheets
#with pd.ExcelWriter('Script_202208221130.xlsx', engine='openpyxl') as writer:
#  for i in range(len(df1_list)-1):
#    df1_list[i].to_excel(writer, sheet_name=f'{df1_list[i].columns[0]}', index=True)
#files.download('Script_202208221130.xlsx')